In [1]:
import numpy as np
import pandas as pd
import warnings
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import json
import tldextract   # Accurately separates a URL's subdomain, domain, and public suffix
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import re
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import nltk 
import spacy
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from wordcloud import WordCloud, ImageColorGenerator,STOPWORDS
import gensim
from gensim.parsing.preprocessing import STOPWORDS as gensim_stopwords
import copy
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords,wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,LancasterStemmer
from sklearn.metrics import f1_score
from pprint import pprint
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', 1000)

[nltk_data] Downloading package stopwords to
[nltk_data]     /gpfs/home/psc21zcu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /gpfs/home/psc21zcu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /gpfs/home/psc21zcu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /gpfs/home/psc21zcu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Loading the ISOT Full Dataset  -- After combined fake and real news  in the previous notebook   "ISOT_ML_Classifiers_TfiDF_Tokeniser"

In [3]:
isot_full_df = pd.read_csv("Updated//ISOT_Combined_FullData.csv")

#### Dropping the columns 'title' , 'text' and 'date'

In [4]:
isot_full_df = isot_full_df.drop(columns = ['title','text', 'subject','date','title_length','body_length','date'])

In [5]:
isot_full_df=isot_full_df.reset_index(drop=True)

In [6]:
isot_full_df.head(2)

,class,fulltext
0,1,"Micheal Moore Demands MI Republican Gov. Be Arrested For Poisoning Flint’s Water (IMAGE) Michael Moore has joined others in calling for U.S. Attorney General Loretta Lynch to investigate Michigan s Republican Governor Rick Snyder for not preventing lead from contaminating the city of Flint s water system. Moore was born and raised in Flint, MI. His first documentary film, Roger & Me, chronicles the downfall of Flint and explains how its demise was orchestrated by big business and conservative policy.Moore posted a photo of himself on social media that shows him holding a pair of handcuffs. In his other hand, Moore is holding up a sign which reads, #ArrestGovSnyder. Activists have been using the hash tag to call attention to Flint s water crisis, and Snyder s culpability in the matter.https://www.facebook.com/mmflint/photos/a.10150288227701857.331023.24674986856/10153223367636857/?type=3&theaterAccompanying the photo, is a letter written by Moore condemning Snyder over the crisis. ..."
1,0,"Trump's son, close associates to appear before Senate President Donald Trump’s son Donald Trump Jr., son-in-law Jared Kushner and former campaign manager Paul Manafort have been asked to appear before U.S. Senate committees next week to answer questions about the campaign’s alleged connections to Russia, officials said on Wednesday. The three men are the closest associates of the president to be called to speak to lawmakers involved in probing Russian meddling in the 2016 U.S. presidential election and possible collusion with the Trump campaign. Trump, who came into office in January, has been dogged by allegations that his campaign officials were connected to Russia, which U.S. intelligence agencies have accused of interfering in last year’s election. Trump has denied any collusion. The U.S. Senate Judiciary Committee said on Wednesday that it had called Trump’s eldest son, Donald Trump Jr., and Manafort to testify on July 26 at a hearing. The president’s son released emails ea..."


In [7]:
# Clean data using the built in cleaner in gensim
isot_full_df['cleantext'] = isot_full_df['fulltext'].apply(lambda x: gensim.utils.simple_preprocess(x))
isot_full_df.head(1)

,class,fulltext,cleantext
0,1,"Micheal Moore Demands MI Republican Gov. Be Arrested For Poisoning Flint’s Water (IMAGE) Michael Moore has joined others in calling for U.S. Attorney General Loretta Lynch to investigate Michigan s Republican Governor Rick Snyder for not preventing lead from contaminating the city of Flint s water system. Moore was born and raised in Flint, MI. His first documentary film, Roger & Me, chronicles the downfall of Flint and explains how its demise was orchestrated by big business and conservative policy.Moore posted a photo of himself on social media that shows him holding a pair of handcuffs. In his other hand, Moore is holding up a sign which reads, #ArrestGovSnyder. Activists have been using the hash tag to call attention to Flint s water crisis, and Snyder s culpability in the matter.https://www.facebook.com/mmflint/photos/a.10150288227701857.331023.24674986856/10153223367636857/?type=3&theaterAccompanying the photo, is a letter written by Moore condemning Snyder over the crisis. ...","[micheal, moore, demands, mi, republican, gov, be, arrested, for, poisoning, flint, water, image, michael, moore, has, joined, others, in, calling, for, attorney, general, loretta, lynch, to, investigate, michigan, republican, governor, rick, snyder, for, not, preventing, lead, from, contaminating, the, city, of, flint, water, system, moore, was, born, and, raised, in, flint, mi, his, first, documentary, film, roger, me, chronicles, the, downfall, of, flint, and, explains, how, its, demise, was, orchestrated, by, big, business, and, conservative, policy, moore, posted, photo, of, himself, on, social, media, that, shows, him, holding, pair, of, handcuffs, in, his, other, hand, moore, is, holding, up, sign, ...]"


### Splitting dataset into train and test

In [8]:
X=isot_full_df['cleantext']
y=isot_full_df['class']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=0)

#### Train the word2vec model from our own dataset

In [10]:
w2v_embedng_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

In [11]:
# We need to make aggregated sentence vectors to replace all words in the sentences with the learned word vectors, 
#then we need to replace each text message with the learned word vectors

words = set(w2v_embedng_model.wv.index_to_key )
X_train_vect = np.array([np.array([w2v_embedng_model.wv[i] for i in m if i in words])
                         for m in X_train])
X_test_vect = np.array([np.array([w2v_embedng_model.wv[i] for i in n if i in words])
                         for n in X_test])

In [12]:
# Average the word vectors for each sentence (and assign a vector of zeros if the model
# did not learn any of the words in the text message during training

X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

#### Helper Functions

In [13]:
import timeit
from timeit import default_timer as timer
from datetime import timedelta
 


def metrics(X_test,y_test,clf):
    predictions=clf.predict(X_test)
    #predictions=(clf.predict_proba(X_test)[:,1] >= 0.3).astype(bool)
    print("confusion_matrix :")
    print(confusion_matrix(y_test,predictions))
    print("Accuracy Score :")
    print(accuracy_score(y_test, predictions))
    print("Classification Report :")
    print(classification_report(y_test, predictions))
    print("F1 score :")
    print(f1_score(y_test, predictions))
    print("ROC AUC Score")
    y_pred_proba = clf.predict_proba(X_test)
    print(roc_auc_score(y_test, y_pred_proba[:,1]) )
    print("------------------------------")

    
def model_comparison_table(X_test,y_test,classifier_list):
    dict_clf={}
    for clf_name,clf in classifier_list:
        predictions=clf.predict(X_test)
        y_pred_proba = clf.predict_proba(X_test)
        accuracy=accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions,average='macro').round(2)
        recall=recall_score(y_test,predictions)
        f1score=f1_score(y_test,predictions).round(2)
        ROC_AUC=roc_auc_score(y_test, y_pred_proba[:,1])
        dict_clf[clf_name]=[accuracy,precision,recall,f1score,ROC_AUC]
    df_models_scores = pd.DataFrame(dict_clf, index=['Accuracy', 'Precision', 'Recall', 'F1 Score','roc_auc_score'])
    return df_models_scores

#### Checking for Random Forest Only using  trained word2vec model

In [15]:
# Instantiate and fit a basic Random Forest model on top of the vectors
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0,n_estimators=800,max_features='auto',max_depth=110,
                            min_samples_split=10,min_samples_leaf=1,bootstrap=False)
rf_model = rf.fit(X_train_vect_avg, y_train.values.ravel())

In [16]:
# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(X_test_vect_avg)
print("Accuracy score : {}".format(accuracy_score(y_test, y_pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, y_pred)))
print("Classification Report")
print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
print("Precision : {}".format(precision))
recall = recall_score(y_test, y_pred)
print("Recall : {}".format(recall))

Accuracy score : 0.9562320355369741
Confusion matrix : 
 [[4054  110]
 [ 225 3265]]
Classification Report
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      4164
           1       0.97      0.94      0.95      3490

    accuracy                           0.96      7654
   macro avg       0.96      0.95      0.96      7654
weighted avg       0.96      0.96      0.96      7654

Precision : 0.9674074074074074
Recall : 0.9355300859598854


### Using pretrained word2vec model

In [14]:
import gensim.downloader

In [15]:
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


##### Download or Load the  pretrained model

In [ ]:
# !wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [16]:
path_to_model = 'Embeddings//GoogleNews-vectors-negative300.bin'

In [17]:
from gensim.models.keyedvectors import KeyedVectors
%time w2v_model = KeyedVectors.load_word2vec_format(path_to_model, binary=True)
print('done loading Word2Vec')

CPU times: user 23.4 s, sys: 1.59 s, total: 25 s
Wall time: 26.5 s
done loading Word2Vec


In [18]:
#Inspect the model
word2vec_vocab = list(w2v_model.index_to_key)

In [19]:
word2vec_vocab_lower = [item.lower() for item in word2vec_vocab]

In [20]:
# Creating a feature vector by averaging all embeddings for all sentences
def embedding_feats(list_of_lists):
    DIMENSION = 300
    zero_vector = np.zeros(DIMENSION)
    feats = []
    for tokens in list_of_lists:
        feat_for_this =  np.zeros(DIMENSION)
        count_for_this = 0 + 1e-5 # to avoid divide-by-zero 
        for token in tokens:
            if token in w2v_model:
                feat_for_this += w2v_model[token]
                count_for_this +=1
        if(count_for_this!=0):
            feats.append(feat_for_this/count_for_this) 
        else:
            feats.append(zero_vector)
    return feats


train_vectors = embedding_feats(X_train)
print(len(train_vectors))

30616


In [21]:
test_vectors = embedding_feats(X_test)
print(len(test_vectors))


7654


### RandomForest

In [24]:
rf = RandomForestClassifier(random_state=0)
rf_model = rf.fit(train_vectors, y_train.values.ravel())

In [25]:
# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, y_pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, y_pred)))
print("Classification Report")
print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
print("Precision : {}".format(precision))
recall = recall_score(y_test, y_pred)
print("Recall : {}".format(recall))

Accuracy score : 0.9314084139012281
Confusion matrix : 
 [[3986  178]
 [ 347 3143]]
Classification Report
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      4164
           1       0.95      0.90      0.92      3490

    accuracy                           0.93      7654
   macro avg       0.93      0.93      0.93      7654
weighted avg       0.93      0.93      0.93      7654

Precision : 0.9464016862390846
Recall : 0.9005730659025788


### Support Vector Machine

In [40]:
svm_clf=SVC(random_state=0,probability=True)

In [41]:
svm_clf.fit(train_vectors,y_train)

pred = svm_clf.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.9659001829108963
Confusion matrix : 
 [[4059  105]
 [ 156 3334]]
Classification Report
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      4164
           1       0.97      0.96      0.96      3490

    accuracy                           0.97      7654
   macro avg       0.97      0.97      0.97      7654
weighted avg       0.97      0.97      0.97      7654



## KNN

In [28]:
#Create KNN Object.
knn_clf = KNeighborsClassifier()

In [29]:
knn_clf.fit(train_vectors,y_train)

pred = knn_clf.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.9344133786255553
Confusion matrix : 
 [[4036  128]
 [ 374 3116]]
Classification Report
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      4164
           1       0.96      0.89      0.93      3490

    accuracy                           0.93      7654
   macro avg       0.94      0.93      0.93      7654
weighted avg       0.94      0.93      0.93      7654



### Xgboost

In [30]:
#Create KNN Object.
xg_clf=XGBClassifier(random_state=0)

xg_clf.fit(train_vectors,y_train)

pred = xg_clf.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))


Accuracy score : 0.9594983015416776
Confusion matrix : 
 [[4049  115]
 [ 195 3295]]
Classification Report
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      4164
           1       0.97      0.94      0.96      3490

    accuracy                           0.96      7654
   macro avg       0.96      0.96      0.96      7654
weighted avg       0.96      0.96      0.96      7654



### Naiva Bayes

In [31]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
nb_clf=GaussianNB()
#p = Pipeline([('Normalizing',MinMaxScaler()),('MultinomialNB',MultinomialNB())])
nb_clf.fit(train_vectors,y_train) 

pred = nb_clf.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.886464593676509
Confusion matrix : 
 [[3745  419]
 [ 450 3040]]
Classification Report
              precision    recall  f1-score   support

           0       0.89      0.90      0.90      4164
           1       0.88      0.87      0.87      3490

    accuracy                           0.89      7654
   macro avg       0.89      0.89      0.89      7654
weighted avg       0.89      0.89      0.89      7654



### Logistic Regression

In [32]:
from sklearn.linear_model import LogisticRegression

In [33]:
logticreg_clf=LogisticRegression()

logticreg_clf.fit(train_vectors,y_train)

pred = logticreg_clf.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.9499608048079435
Confusion matrix : 
 [[4000  164]
 [ 219 3271]]
Classification Report
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      4164
           1       0.95      0.94      0.94      3490

    accuracy                           0.95      7654
   macro avg       0.95      0.95      0.95      7654
weighted avg       0.95      0.95      0.95      7654



### Consolidating all classifiers accuracy scores

In [42]:
classifiers = [('Logistic Regression',logticreg_clf),('Naive Bayes',nb_clf),('Xgboost',xg_clf),
              ('KNN',knn_clf),('Random Forest',rf_model),("SVM",svm_clf)]

In [43]:
df_model_scores=model_comparison_table(test_vectors,y_test,classifiers)
df_model_scores.head(20).T.sort_values(by='Accuracy', ascending=False)

,Accuracy,Precision,Recall,F1 Score,roc_auc_score
SVM,0.965900,0.97,0.955301,0.96,0.994732
Xgboost,0.959498,0.96,0.944126,0.96,0.993060
Logistic Regression,0.949961,0.95,0.937249,0.94,0.989200
KNN,0.934413,0.94,0.892837,0.93,0.976047
Random Forest,0.931408,0.93,0.900573,0.92,0.980619
Naive Bayes,0.886465,0.89,0.871060,0.87,0.944141


## Pretrained Glove

#### Download the zip file

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip

#### Unzip it

In [ ]:
# !unzip glove*.zip

In [45]:
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [46]:
# Load 300 dimension Glove pretrained model "glove.6B.300d.txt" and convert to word2vec format",
path_to_model = '//gpfs//home//psc21zcu//FakeNews_Detection/Dissertation_Work//Embeddings//glove.6B.300d.txt'
output_file = '//gpfs//home//psc21zcu//FakeNews_Detection/Dissertation_Work//Embeddings///gensim_glove.6B.300d.txt'
glove2word2vec(path_to_model, output_file)

(400000, 300)

In [47]:
path_to_model

'//gpfs//home//psc21zcu//FakeNews_Detection/Dissertation_Work//Embeddings//glove.6B.300d.txt'

In [48]:
%time glove_model = KeyedVectors.load_word2vec_format(output_file, binary=False)
print('done loading Glove Pretrained Embedding')

CPU times: user 54.8 s, sys: 846 ms, total: 55.6 s
Wall time: 55.9 s
done loading Glove Pretrained Embedding


In [49]:
# https://colab.research.google.com/github/practical-nlp/practical-nlp/blob/master/Ch4/03_Word2Vec_Example.ipynb#scrollTo=m-WjFyC6b_IE

# Creating a feature vector by averaging all embeddings for all sentences
def glove_embedding_feats(list_of_lists):
    DIMENSION = 300
    zero_vector = np.zeros(DIMENSION)
    feats = []
    for tokens in list_of_lists:
        feat_for_this =  np.zeros(DIMENSION)
        count_for_this = 0 + 1e-5 # to avoid divide-by-zero 
        for token in tokens:
            if token in glove_model:
                feat_for_this += glove_model[token]
                count_for_this +=1
        if(count_for_this!=0):
            feats.append(feat_for_this/count_for_this) 
        else:
            feats.append(zero_vector)
    return feats


glove_train_vectors = glove_embedding_feats(X_train)
print(len(glove_train_vectors))

30616


In [50]:
glove_test_vectors = glove_embedding_feats(X_test)
print(len(glove_test_vectors))

7654


### RandomForest

In [53]:
rf = RandomForestClassifier(random_state=0)
rf_clf_g = rf.fit(glove_train_vectors, y_train.values.ravel())

In [54]:
# Use the trained model to make predictions on the test data
y_pred = rf_clf_g.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, y_pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, y_pred)))
print("Classification Report")
print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
print("Precision : {}".format(precision))
recall = recall_score(y_test, y_pred)
print("Recall : {}".format(recall))

Accuracy score : 0.932322968382545
Confusion matrix : 
 [[3979  185]
 [ 333 3157]]
Classification Report
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      4164
           1       0.94      0.90      0.92      3490

    accuracy                           0.93      7654
   macro avg       0.93      0.93      0.93      7654
weighted avg       0.93      0.93      0.93      7654

Precision : 0.9446439257929383
Recall : 0.9045845272206303


### Support Vector Machine

In [55]:
svm_clf_g=SVC(random_state=0,probability=True)

In [56]:
svm_clf_g.fit(glove_train_vectors,y_train)

pred = svm_clf_g.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.959890253462242
Confusion matrix : 
 [[4017  147]
 [ 160 3330]]
Classification Report
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      4164
           1       0.96      0.95      0.96      3490

    accuracy                           0.96      7654
   macro avg       0.96      0.96      0.96      7654
weighted avg       0.96      0.96      0.96      7654



## KNN

In [58]:
#Create KNN Object.
knn_clf_g = KNeighborsClassifier()

In [59]:
knn_clf_g.fit(glove_train_vectors,y_train)

pred = knn_clf_g.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.9358505356676248
Confusion matrix : 
 [[4021  143]
 [ 348 3142]]
Classification Report
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      4164
           1       0.96      0.90      0.93      3490

    accuracy                           0.94      7654
   macro avg       0.94      0.93      0.93      7654
weighted avg       0.94      0.94      0.94      7654



### Xgboost

In [60]:
#Create KNN Object.
xg_clf_g=XGBClassifier(random_state=0)

xg_clf_g.fit(glove_train_vectors,y_train)

pred = xg_clf_g.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))


Accuracy score : 0.9602822053828064
Confusion matrix : 
 [[4027  137]
 [ 167 3323]]
Classification Report
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      4164
           1       0.96      0.95      0.96      3490

    accuracy                           0.96      7654
   macro avg       0.96      0.96      0.96      7654
weighted avg       0.96      0.96      0.96      7654



### Naiva Bayes

In [61]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
nb_clf_g=GaussianNB()

nb_clf_g.fit(glove_train_vectors,y_train) 

pred = nb_clf_g.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.8808466161484191
Confusion matrix : 
 [[3670  494]
 [ 418 3072]]
Classification Report
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      4164
           1       0.86      0.88      0.87      3490

    accuracy                           0.88      7654
   macro avg       0.88      0.88      0.88      7654
weighted avg       0.88      0.88      0.88      7654



### Logistic Regression

In [62]:
from sklearn.linear_model import LogisticRegression

In [63]:
logticreg_clf_g=LogisticRegression()

logticreg_clf_g.fit(glove_train_vectors,y_train)

pred = logticreg_clf_g.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.9584530964201725
Confusion matrix : 
 [[4011  153]
 [ 165 3325]]
Classification Report
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      4164
           1       0.96      0.95      0.95      3490

    accuracy                           0.96      7654
   macro avg       0.96      0.96      0.96      7654
weighted avg       0.96      0.96      0.96      7654



### Consolidating all classifiers accuracy scores

In [64]:
classifiers = [('Logistic Regression',logticreg_clf_g),('Naive Bayes',nb_clf_g),('Xgboost',xg_clf_g),
              ('KNN',knn_clf_g),('Random Forest',rf_clf_g),("SVM",svm_clf_g)]

In [65]:
df_model_scores=model_comparison_table(glove_test_vectors,y_test,classifiers)
df_model_scores.head(20).T.sort_values(by='Accuracy', ascending=False)

,Accuracy,Precision,Recall,F1 Score,roc_auc_score
Xgboost,0.960282,0.96,0.952149,0.96,0.992769
SVM,0.959890,0.96,0.954155,0.96,0.991847
Logistic Regression,0.958453,0.96,0.952722,0.95,0.991594
KNN,0.935851,0.94,0.900287,0.93,0.975219
Random Forest,0.932323,0.93,0.904585,0.92,0.981253
Naive Bayes,0.880847,0.88,0.880229,0.87,0.935085
